In [1]:
import sqlite3, csv
import numpy as np
from scipy import stats
import pandas as pd

In [2]:
conn = sqlite3.connect('sql_fang.db')
conn.text_factory = str
c = conn.cursor()

In [3]:
c.execute("DROP TABLE IF EXISTS A;")
c.execute('''CREATE TABLE A AS SELECT ZIPCODE, SCORE FROM WebExtract_f
                ;''')
a = c.fetchall()
#print a

In [4]:
class ScipySEMAggregate(object):
    def __init__(self):
        self.values = []
    def step(self, value):
        #print self.values
        if value != '':
            self.values.append(float(value))
    def finalize(self):
        return stats.sem(self.values)
conn.create_aggregate("sp_sem", 1, ScipySEMAggregate)

In [5]:
c.execute("DROP TABLE IF EXISTS B;")
c.execute('''CREATE TABLE B AS SELECT ZIPCODE, AVG(SCORE) AS MEANGRADE, 
                    sp_sem(SCORE) AS SEM, COUNT(*) AS NUM_INSP FROM WebExtract_f
                    GROUP BY ZIPCODE
                    ;''')

/opt/conda/lib/python2.7/site-packages/numpy/core/_methods.py:83: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [6]:
c.execute("DROP TABLE IF EXISTS C;")
c.execute('''CREATE TABLE C AS SELECT * FROM B
                WHERE NUM_INSP>100
                ;''')

In [7]:
c.execute('''SELECT * FROM C
                    ;''')
a = c.fetchall()


In [8]:
len(a)

184

In [9]:
a[0]

('10001', 19.750465753424656, 0.16172391617084478, 9125)

# Boro

In [10]:
c.execute("DROP TABLE IF EXISTS FANG2;")
c.execute('''CREATE TABLE FANG2 AS SELECT BORO, AVG(SCORE) AS MEANGRADE, 
                    sp_sem(SCORE) AS SEM, COUNT(*) AS NUM_INSP FROM WebExtract_f
                    GROUP BY BORO
                    ;''')

In [11]:
c.execute('''SELECT * FROM FANG2
             WHERE BORO>0
             ORDER BY MEANGRADE
                    ;''')
a = c.fetchall()


In [12]:
df = pd.DataFrame(a,columns=['a','b', 'c', 'd']).sort('b')
df

,a,b,c,d
0,5,19.372829,0.107063,17386
1,2,20.220329,0.071172,48641
2,3,20.689263,0.044395,125080
3,1,20.900152,0.033281,217231
4,4,21.252587,0.045651,123494


In [13]:
sum(df.d)

531832

In [14]:
c.execute('''UPDATE FANG2
                    SET BORO='MANHATTAN'
                    WHERE BORO='1'
                    ;''')
c.execute('''UPDATE FANG2
                    SET BORO='THE BRONX'
                    WHERE BORO='2'
                    ;''')
c.execute('''UPDATE FANG2
                    SET BORO='BROOKLYN'
                    WHERE BORO='3'
                    ;''')
c.execute('''UPDATE FANG2
                    SET BORO='QUEENS'
                    WHERE BORO='4'
                    ;''')
c.execute('''UPDATE FANG2
                    SET BORO='STATEN ISLAND'
                    WHERE BORO='5'
                    ;''')

In [15]:
c.execute('''SELECT * FROM FANG2
             WHERE BORO>0
             ORDER BY MEANGRADE
                    ;''')
a = c.fetchall()


In [16]:
a

[('STATEN ISLAND', 19.37282871275739, 0.10706295846600054, 17386),
 ('THE BRONX', 20.220328529429906, 0.071172271471837, 48641),
 ('BROOKLYN', 20.68926287176207, 0.04439474801785955, 125080),
 ('MANHATTAN', 20.90015237235938, 0.033280690323082764, 217231),
 ('QUEENS', 21.252587170226892, 0.04565104618156244, 123494)]

# Cuisian

In [86]:
c.execute("DROP TABLE IF EXISTS FANG3;")
c.execute('''CREATE TABLE FANG3 AS SELECT CUISINECODE, AVG(SCORE) AS MEANGRADE, 
                    sp_sem(SCORE) AS SEM, COUNT(*) AS NUM_INSP FROM WebExtract_f
                    GROUP BY CUISINECODE
                    ;''')

In [87]:
c.execute('''SELECT * FROM FANG3
             WHERE NUM_INSP>100
             ORDER BY MEANGRADE
                    ;''')
a = c.fetchall()


In [88]:
c.execute("DROP TABLE IF EXISTS FANG32;")
c.execute('''CREATE TABLE FANG32 AS SELECT Fang3.*, CUISINE_f.CODEDESC
                FROM FANG3
                LEFT JOIN CUISINE_f
                ON Fang3.CUISINECODE=CUISINE_f.CUISINECODE
                ;''')

In [189]:
c.execute('''SELECT CUISINECODE, MEANGRADE, SEM, NUM_INSP FROM FANG32
             WHERE NUM_INSP>100
             ORDER BY MEANGRADE
                    ;''')
a = c.fetchall()


In [190]:
a

[('Creole', 28.96069868995633, 0.8280390563340794, 687),
 ('Chinese/Cuban', 26.248306997742663, 0.8218178838154246, 443),
 ('Bangladeshi', 25.443488943488944, 0.6541898062400501, 814),
 ('African', 25.394778902503997, 0.4548654080746962, 1877),
 ('Chinese/Japanese', 25.0666161998486, 0.4581240303898391, 1321),
 ('Asian', 24.586526099254307, 0.21557599197048327, 7778),
 ('Pakistani', 24.266265060240965, 0.6415148865470445, 830),
 ('Chinese', 23.942464820699048, 0.06816574861933593, 61684),
 ('Indian', 23.86602409638554, 0.2024034646705623, 8300),
 ('Peruvian', 23.529473135106937, 0.3972456021278627, 1917),
 ('Vietnamese/Cambodian/Malaysia',
  23.448814655172413,
  0.3722490747522786,
  1856),
 ('Delicatessen', 23.247045524612066, 0.16235492849437935, 9731),
 ('Korean', 23.086021505376344, 0.19530237824845778, 7068),
 ('Portuguese', 22.940350877192984, 1.1079022099407372, 285),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
  22.92815885839405,
  0.1027774699594768

In [96]:
len(a)

75

# Violation

In [135]:
c.execute("DROP TABLE IF EXISTS FANG41;")
c.execute('''CREATE TABLE FANG41 AS SELECT VIOLATIONCODE, VIOLATIONDESC FROM VIOLATION_f
                    WHERE CAST(ENDDATE AS DATE)>CAST('2014-01-01 00:00:00' AS DATE)
                    ;''')

c.execute("DROP TABLE IF EXISTS FANG42;")
c.execute('''CREATE TABLE FANG42 AS SELECT VIOLCODE, CUISINECODE FROM WebExtract_f
                    ;''')

In [251]:
c.execute('''SELECT * FROM FANG41
                    ;''')
a = c.fetchall()
a

[('02A', 'Food not cooked to required minimum temperature.'),
 ('02B', 'Hot food item not held at or above 140\xba F.'),
 ('02C',
  'Hot food item that has been cooked and refrigerated is being held for service without first being reheated to 1 65\xba F or above within 2 hours.'),
 ('02D',
  'Precooked potentially hazardous food from commercial food processing establishment that is supposed to be heated, but is not heated to 140\xba F within 2 hours.'),
 ('02E',
  'Whole frozen poultry or poultry breasts, other than a single portion, is being cooked frozen or partially thawed.'),
 ('02F',
  'Meat, fish or molluscan shellfish served raw or undercooked without prior notification to customer.'),
 ('02G',
  'Cold food item held above 41\xba F (smoked fish and reduced oxygen packaged foods above 38 \xbaF) except during necessary preparation.'),
 ('02H',
  'Food not cooled by an approved method whereby the internal product temperature is reduced from 140\xba F to 70\xba F or less within 2 ho

Calculate uncond probability

TOTAL violation number 

In [340]:
c.execute('''SELECT COUNT(*) FROM WebExtract_f''')
a = c.fetchall()[0]
N = a[0]
N

531935

In [358]:
c.execute("DROP TABLE IF EXISTS FangUN0;")
c.execute('''CREATE TABLE FangUN0 AS 
            SELECT COUNT(*)*1.0/531935 AS CNT, VIOLCODE FROM WebExtract_f
                GROUP BY VIOLCODE
                    ;''')
c.execute('''SELECT * FROM FangUN0 ORDER BY CNT DESC''')
a = c.fetchall()
a

[(0.12486488010753194, '10F'),
 (0.09317679791703874, '08A'),
 (0.08613646404165923, '02G'),
 (0.07201255792531042, '04L'),
 (0.05890005357797475, '10B'),
 (0.05821387951535432, '06D'),
 (0.047693797174466805, '06C'),
 (0.04610149736339966, '02B'),
 (0.04043351161326102, '04N'),
 (0.029992386287798323, '04H'),
 (0.02738680477877936, '04M'),
 (0.025832103546485942, '06E'),
 (0.02108340304736481, ''),
 (0.018829368249880154, '06F'),
 (0.01763561337381447, '06A'),
 (0.017415661687988193, '04A'),
 (0.01640613984791375, '10H'),
 (0.012794796356697717, '09C'),
 (0.01179091430343933, '04J'),
 (0.011730756577401373, '08C'),
 (0.011204376474569261, '05D'),
 (0.01101638358070065, '10A'),
 (0.011001344149191161, '16B'),
 (0.00936016618571818, '10D'),
 (0.008884544164230592, '04C'),
 (0.008273567259157604, '22C'),
 (0.008204009888426218, '09B'),
 (0.007327963002998487, '10I'),
 (0.007096731743540094, '20F'),
 (0.006647428727194112, '10E'),
 (0.006215045071296305, '04K'),
 (0.006055251111507985, '0

In [17]:
A

NameError: name 'A' is not defined

In [303]:
c.execute("DROP TABLE IF EXISTS Fang43A;")
c.execute('''CREATE TABLE Fang43A AS  Select CUISINECODE, VIOLCODE, Count(*) As Count1
                From   FANG42
                Group By CUISINECODE, VIOLCODE
                ;''')

In [304]:
c.execute("DROP TABLE IF EXISTS Fang43AA;")
c.execute('''CREATE TABLE Fang43AA AS SELECT * FROM Fang43A
             WHERE Count1>100
                    ;''')
c.execute('''SELECT * FROM Fang43AA''')
a = c.fetchall()
a

[('02', '02B', 142),
 ('02', '02G', 113),
 ('02', '04L', 175),
 ('02', '04N', 114),
 ('02', '08A', 224),
 ('02', '10F', 197),
 ('03', '', 2914),
 ('03', '02A', 130),
 ('03', '02B', 3836),
 ('03', '02C', 110),
 ('03', '02G', 10890),
 ('03', '02H', 673),
 ('03', '03C', 143),
 ('03', '04A', 2195),
 ('03', '04C', 1375),
 ('03', '04H', 5779),
 ('03', '04J', 1473),
 ('03', '04K', 758),
 ('03', '04L', 8351),
 ('03', '04M', 2286),
 ('03', '04N', 5725),
 ('03', '04O', 138),
 ('03', '05D', 1527),
 ('03', '05F', 142),
 ('03', '05H', 190),
 ('03', '06A', 1810),
 ('03', '06B', 673),
 ('03', '06C', 5379),
 ('03', '06D', 9488),
 ('03', '06E', 3113),
 ('03', '06F', 2022),
 ('03', '08A', 11126),
 ('03', '08B', 517),
 ('03', '08C', 1516),
 ('03', '09A', 677),
 ('03', '09B', 611),
 ('03', '09C', 1646),
 ('03', '10A', 1438),
 ('03', '10B', 8246),
 ('03', '10C', 106),
 ('03', '10D', 966),
 ('03', '10E', 860),
 ('03', '10F', 16562),
 ('03', '10H', 2744),
 ('03', '10I', 973),
 ('03', '10J', 820),
 ('03', '15

Calculate conditional probability

In [305]:
c.execute("DROP TABLE IF EXISTS Fang43;")
c.execute('''CREATE TABLE Fang43 AS Select A.VIOLCODE, A.CUISINECODE, A.Count1 * 1.0 / B.Count2 As Freq, A.Count1
             From Fang43AA As A
        Inner Join (
                Select CUISINECODE, Count(*) As Count2
                From   FANG42
                Group By CUISINECODE
                ) As B
                On A.CUISINECODE = B.CUISINECODE
                ORDER BY Freq DESC
                ;''')

In [350]:
c.execute('''SELECT * FROM Fang43 ;''')
a = c.fetchall()


In [307]:
a

[('', '99', 0.32362459546925565, 900),
 ('10F', '29', 0.2071639013800589, 1336),
 ('10F', '39', 0.19458566344687342, 1531),
 ('10F', '75', 0.18518518518518517, 110),
 ('02G', '75', 0.17845117845117844, 106),
 ('10F', '14', 0.17131604857236626, 2610),
 ('10F', '69', 0.17111978384869408, 1140),
 ('10F', '43', 0.16961690885072656, 642),
 ('10F', '70', 0.158096454806771, 495),
 ('10F', '18', 0.14885796797059594, 1134),
 ('10F', '07', 0.14868491680085882, 554),
 ('10F', '51', 0.14164827078734363, 385),
 ('10F', '47', 0.13365788891347644, 604),
 ('10F', '10', 0.1323529411764706, 108),
 ('10F', '12', 0.13200379867046533, 139),
 ('10F', '84', 0.13161875945537066, 261),
 ('10F', '03', 0.12901567319976318, 16562),
 ('10F', '62', 0.12783993279621214, 3348),
 ('10F', '73', 0.12776176753121998, 133),
 ('10F', '81', 0.1275191104933982, 367),
 ('02G', '27', 0.1267084575069366, 1233),
 ('10F', '50', 0.12421301154249738, 947),
 ('10F', '27', 0.12362552666735176, 1203),
 ('10F', '78', 0.1222161868549701

Add in uncon probability, FangUN0 AS 
            SELECT COUNT(*)*1.0/531935 AS CNT, VIOLCODE FROM WebExtract_f

In [373]:
c.execute("DROP TABLE IF EXISTS FANG0;")
c.execute('''CREATE TABLE FANG0 AS SELECT Fang43.Freq*1.0/FangUN0.CNT AS Prob, * FROM FANG43
                LEFT JOIN FangUN0
                ON Fang43.VIOLCODE=FangUN0.VIOLCODE
                ORDER BY PROB DESC
                ;''')

In [374]:
c.execute('''SELECT * FROM Fang0;''')
a = c.fetchall()
a

[(15.349732428973562,
  '',
  '99',
  0.32362459546925565,
  900,
  0.02108340304736481,
  ''),
 (3.265525983435347,
  '04C',
  '49',
  0.02901270981927388,
  541,
  0.008884544164230592,
  '04C'),
 (3.1414818233282684,
  '20D',
  '14',
  0.011486708237610764,
  175,
  0.0036564617857444987,
  '20D'),
 (3.063222689585581,
  '04A',
  '51',
  0.05334805003679176,
  145,
  0.017415661687988193,
  '04A'),
 (3.0324753951821077,
  '10E',
  '29',
  0.02015816405644286,
  130,
  0.006647428727194112,
  '10E'),
 (2.927867679436372,
  '04A',
  '43',
  0.05099075297225892,
  193,
  0.017415661687988193,
  '04A'),
 (2.6494362448587756,
  '09B',
  '82',
  0.02173600115157622,
  151,
  0.008204009888426218,
  '09B'),
 (2.368381554925711,
  '04H',
  '47',
  0.07103341447222837,
  321,
  0.029992386287798323,
  '04H'),
 (2.329632394606289,
  '02H',
  '55',
  0.01383935700218236,
  260,
  0.005940575446248132,
  '02H'),
 (2.271493060850999,
  '02H',
  '44',
  0.013493975903614458,
  112,
  0.0059405754

In [375]:
c.execute("DROP TABLE IF EXISTS FANGfinal;")
c.execute('''CREATE TABLE FANGfinal AS SELECT A.*, Fang41.VIOLATIONDESC AS VIOlationdesc FROM 
                (SELECT Fang0.*, CUISINE_f.CODEDESC
                FROM FANG0
                LEFT JOIN CUISINE_f
                ON Fang0.CUISINECODE=CUISINE_f.CUISINECODE) AS A
                LEFT JOIN Fang41
                ON A.VIOLCODE=Fang41.VIOLATIONCODE
                ;''')

In [388]:
c.execute('''SELECT CODEDESC, VIOLATIONDESC, PROB, Count1 FROM FANGfinal;''')
a = c.fetchall()
a[1:21]

[('Japanese',
  'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.',
  3.265525983435347,
  541),
 ('Caf\xe9/Coffee/Tea',
  '\x93Choking first aid\x94 poster not posted. \x93Alcohol and pregnancy\x94 warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.',
  3.1414818233282684,
  175),
 ('Juice, Smoothies, Fruit Salads',
  'Food Protection Certificate not held by supervisor of food operations.',
  3.063222689585581,
  145),
 ('Donuts',
  'Accurate thermometer not provided in refrigerated or hot holding equipment.',
  3.0324753951821077,
  130),
 ('Ice Cream, Gelato, Yogurt, Ices',
  'Food Protection Certificate not held by supervisor of food operations.',
  2.927867679436372,
  193),
 ('Thai', 'Thawing procedures improper.', 2.6494362448587756, 151),
 ('Irish',
  'Raw, cooked or prepare

In [390]:
lt = [((k[0], k[1]), k[2], k[3]) for k in a[1:21]]
lt

[(('Japanese',
   'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.'),
  3.265525983435347,
  541),
 (('Caf\xe9/Coffee/Tea',
   '\x93Choking first aid\x94 poster not posted. \x93Alcohol and pregnancy\x94 warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.'),
  3.1414818233282684,
  175),
 (('Juice, Smoothies, Fruit Salads',
   'Food Protection Certificate not held by supervisor of food operations.'),
  3.063222689585581,
  145),
 (('Donuts',
   'Accurate thermometer not provided in refrigerated or hot holding equipment.'),
  3.0324753951821077,
  130),
 (('Ice Cream, Gelato, Yogurt, Ices',
   'Food Protection Certificate not held by supervisor of food operations.'),
  2.927867679436372,
  193),
 (('Thai', 'Thawing procedures improper.'), 2.6494362448587756, 151),
 (('Irish',
   'Raw

In [383]:
tt = [k[2] for k in a]

In [387]:
np.array(tt[1:21]).mean()

2.3700921634985899